<a href="https://colab.research.google.com/github/AravindReddy123/Aravind_INFO5731_Spring2023/blob/main/Assignment_Four_INFO5731_Assignment_Four_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Four**

In this assignment, you are required to conduct topic modeling, sentiment analysis based on **the dataset you created from assignment three**.

# **Question 1: Topic Modeling**

(30 points). This question is designed to help you develop a feel for the way topic modeling works, the connection to the human meanings of documents. Based on the dataset from assignment three, write a python program to **identify the top 10 topics in the dataset**. Before answering this question, please review the materials in lesson 8, especially the code for LDA, LSA, and BERTopic. The following information should be reported:

(1) Features (text representation) used for topic modeling.

(2) Top 10 clusters for topic modeling.

(3) Summarize and describe the topic for each cluster. 


In [1]:
# Write your code here
# Write your code here

import requests
import json
import pandas as pd
import time
from google.colab import  drive
drive.mount('/drive', force_remount=True)
#Initializing the limit to 100 so that for everytime we hit the API we get only 100 records.
limit="100"
#Declaring the URL
url = "https://api.semanticscholar.org/graph/v1/paper/search?query=machine%20learning+data%20science+artificial%20intelligence&fields=abstract&offset="
appended_data = []
for i in range(0,1000,100):
  offset = str(i)
  #Creating the dynamic url with offset changing for every iteration of loop
  dynamic_url = url+offset+"&limit="+limit
  time.sleep(5)
  #Calling get method from request library and sending url as parameter in get method to get the required data from API.
  response = requests.get(dynamic_url,headers={"x-api-key":"wMeN1WdWq75wh1hHr5CM34dDRJCaS5MQFD6s67kb"})
  #Converting the text response we got from the above to JSON format
  print(response.text)
  data_response = json.loads(response.text)['data']
  #Converting the JSON data format to pandas Dataframe.
  df = pd.DataFrame(data_response)
  #df = df.dropna(axis=0,subset = ['abstract'])
  #concatenating the dataframes for each offset to appended_data variable
  appended_data.append(df)
#print(appended_data[0])
appended_data = pd.concat(appended_data,ignore_index=True)
appended_data = pd.DataFrame(appended_data,columns=['paperId','abstract'])
appended_data = appended_data.dropna(axis=0,subset = ['abstract'])


Mounted at /drive
{"total": 102806, "offset": 0, "next": 100, "data": [{"paperId": "1e85606f75af68884122f0d1d8393c000e915194", "abstract": "This Special Issue of the IEEE Transactions on Plasma Science (TPS) follows the first American Physical Society Division of Plasma Physics (APS-DPP) mini-conference on Machine Learning, Data Science, and Artificial Intelligence in Plasma Research held during the 60th APS-DPP Meeting in Portland, OR, USA (November 5\u20139, 2018). It contains selected highlights from not only the mini-conference but also the broader plasma physics community. Although data science has a long and rich history in plasma physics, dating back at least three decades, it is experiencing a renaissance, thanks in large part to the advances outside of plasma physics. Novel algorithms, hardware, and analytic techniques (buoyed by the open source software ecosystem) have led plasma scientists to explore ways in which the data revolution could accelerate and inform scientific di

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
# Convert the text data into a bag of words representation
vectorizer = CountVectorizer(stop_words='english')
doc_word_matrix = vectorizer.fit_transform(appended_data['abstract'])
# Fit the LDA model to the data
lda_model = LatentDirichletAllocation(n_components=10, random_state=0)
lda_model.fit(doc_word_matrix)
# Print the top 10 words in each topic
for topic_idx, topic in enumerate(lda_model.components_):
    print("Topic #%d:" % topic_idx)
    print(" ".join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]]))
    print()

# Cluster the documents
kmeans_model = KMeans(n_clusters=10, random_state=0)
doc_topic_matrix = lda_model.transform(doc_word_matrix)
kmeans_model.fit(doc_topic_matrix)

# Summarize and describe the topics for each cluster
for cluster_idx in range(10):
    print("Cluster #%d:" % cluster_idx)
    cluster_docs = appended_data[kmeans_model.labels_ == cluster_idx]
    cluster_topics = doc_topic_matrix[kmeans_model.labels_ == cluster_idx].mean(axis=0)
    top_topics = cluster_topics.argsort()[:-11:-1]
    for topic_idx in top_topics:
        print("Topic #%d: %f" % (topic_idx, cluster_topics[topic_idx]))
        print(" ".join([vectorizer.get_feature_names_out()[i] for i in lda_model.components_[topic_idx].argsort()[:-11:-1]]))
    print()

Topic #0:
data ai ml science learning research artificial machine intelligence health

Topic #1:
data learning machine artificial intelligence materials algorithms models systems design

Topic #2:
data mining fusion learning analysis machine algorithms using methods used

Topic #3:
data ai learning intelligence security artificial machine cancer information science

Topic #4:
intelligence ai learning artificial machine science data human computer systems

Topic #5:
systems data intelligence human medical field artificial new algorithms big

Topic #6:
science systems data intelligence machine learning intelligent networks methods ml

Topic #7:
ai data learning intelligence artificial machine research science health deep

Topic #8:
intelligence artificial data machine new learning science social performance technologies

Topic #9:
learning data machine science intelligence artificial ai research deep computer



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster #0:
Topic #4: 0.851462
intelligence ai learning artificial machine science data human computer systems
Topic #0: 0.048671
data ai ml science learning research artificial machine intelligence health
Topic #1: 0.042033
data learning machine artificial intelligence materials algorithms models systems design
Topic #9: 0.041038
learning data machine science intelligence artificial ai research deep computer
Topic #3: 0.005494
data ai learning intelligence security artificial machine cancer information science
Topic #6: 0.004557
science systems data intelligence machine learning intelligent networks methods ml
Topic #7: 0.004031
ai data learning intelligence artificial machine research science health deep
Topic #5: 0.001050
systems data intelligence human medical field artificial new algorithms big
Topic #2: 0.000832
data mining fusion learning analysis machine algorithms using methods used
Topic #8: 0.000832
intelligence artificial data machine new learning science social performance

In [3]:
from textblob import TextBlob

# Define a function to get sentiment score
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        return 1
    elif sentiment == 0:
        return 0
    else:
        return -1

# Apply the function to the DataFrame column and store the results in a new column
appended_data['sentiment'] = appended_data['abstract'].apply(get_sentiment)

In [4]:
appended_data

,paperId,abstract,sentiment
0,1e85606f75af68884122f0d1d8393c000e915194,This Special Issue of the IEEE Transactions on...,1
2,72d3ddf1f7210d7e70144bbc09f770ec411fe909,Smarter applications are making better use of ...,1
3,a20c39f5439b84b57d883a21b698394d2347800d,"Over the years, many clinical and engineering ...",-1
4,ddb9ba8651e39d3c2e7a990c4b27049c20aff228,Machine learning (ML) is becoming capable of t...,1
9,12d89245440d8c2a57c0741d10177189adf230d3,Abstract The Petaflops supercomputer “Zhores” ...,1
...,...,...,...
995,ab95f9e5697a7619da9b4a8dbf81f338f2b2ec21,We consider the use of intelligent systems to ...,1
996,c59f6b9b3644bd776433828aa88c6c0d00f42622,Expert systems are part of a general category ...,1
997,9fd6621c52fbfff2778448515a83df000358597e,"In 1989, B. F. Skinner told Joseph Rychlak tha...",1
998,55d0b6681645c91a4436a36d9aec4a1b3f542e97,Carbon nanotubes have an exciting array of app...,1


# **Question 2: Sentiment Analysis**

(30 points). Sentiment analysis also known as opinion mining is a sub field within Natural Language Processing (NLP) that builds machine learning algorithms to classify a text according to the sentimental polarities of opinions it contains, e.g., positive, negative, neutral. The purpose of this question is to develop a machine learning classifier for sentiment analysis. Based on the dataset from assignment three, write a python program to implement a sentiment classifier and evaluate its performance. Notice: **80% data for training and 20% data for testing**.  

(1) Features used for sentiment classification and explain why you select these features.

(2) Select two of the supervised learning algorithm from scikit-learn library: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning, to build a sentiment classifier respectively. Note: Cross-validation (5-fold or 10-fold) should be conducted. Here is the reference of cross-validation: https://scikit-learn.org/stable/modules/cross_validation.html.

(3) Compare the performance over accuracy, precision, recall, and F1 score for the two algorithms you selected. Here is the reference of how to calculate these metrics: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9. 

In [5]:
# Write your code here

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [6]:
# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(appended_data['abstract'], appended_data['sentiment'], test_size=0.2, random_state=0)

In [7]:
# Create a bag of words representation of the data
vectorizer = CountVectorizer(stop_words='english')
train_features = vectorizer.fit_transform(train_data)
test_features = vectorizer.transform(test_data)

In [8]:
# Train a Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_cv_scores = cross_val_score(nb_classifier, train_features, train_labels, cv=5)
nb_classifier.fit(train_features, train_labels)
nb_predictions = nb_classifier.predict(test_features)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [9]:
# Train a Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)
lr_cv_scores = cross_val_score(lr_classifier, train_features, train_labels, cv=5)
lr_classifier.fit(train_features, train_labels)
lr_predictions = lr_classifier.predict(test_features)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [10]:
# Evaluate the performance of the classifiers
print("Multinomial Naive Bayes")
print("Accuracy: %.3f" % accuracy_score(test_labels, nb_predictions))
print("Precision: %.3f" % precision_score(test_labels, nb_predictions, average='weighted'))
print("Recall: %.3f" % recall_score(test_labels, nb_predictions, average='weighted'))
print("F1 Score: %.3f" % f1_score(test_labels, nb_predictions, average='weighted'))
print("Cross-Validation Scores:", nb_cv_scores)
print("Average Cross-Validation Score: %.3f" % nb_cv_scores.mean())
print()

Multinomial Naive Bayes
Accuracy: 0.787
Precision: 0.763
Recall: 0.787
F1 Score: 0.710
Cross-Validation Scores: [0.76612903 0.77419355 0.77419355 0.75806452 0.78225806]
Average Cross-Validation Score: 0.771



In [11]:
print("Logistic Regression")
print("Accuracy: %.3f" % accuracy_score(test_labels, lr_predictions))
print("Precision: %.3f" % precision_score(test_labels, lr_predictions, average='weighted'))
print("Recall: %.3f" % recall_score(test_labels, lr_predictions, average='weighted'))
print("F1 Score: %.3f" % f1_score(test_labels, lr_predictions, average='weighted'))
print("Cross-Validation Scores:", lr_cv_scores)
print("Average Cross-Validation Score: %.3f" % lr_cv_scores.mean())

Logistic Regression
Accuracy: 0.813
Precision: 0.794
Recall: 0.813
F1 Score: 0.796
Cross-Validation Scores: [0.81451613 0.84677419 0.81451613 0.79032258 0.82258065]
Average Cross-Validation Score: 0.818


# **Question 3: House price prediction**

(40 points). You are required to build a **regression** model to predict the house price with 79 explanatory variables describing (almost) every aspect of residential homes. The purpose of this question is to practice regression analysis, an supervised learning model. The training data, testing data, and data description files can be download from canvas. Here is an axample for implementation: https://towardsdatascience.com/linear-regression-in-python-predict-the-bay-areas-home-price-5c91c8378878. 


In [32]:
# Write your code here
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv('/content/drive/MyDrive/assignment4-question3-data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/assignment4-question3-data/test.csv')

# Fill missing values
train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)
# create a label encoder object
le = LabelEncoder()
# Preprocessing
train.drop(['Id'], axis=1, inplace=True)  # Remove Id column
test.drop(['Id'], axis=1, inplace=True)
train[['MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition']]=train[['MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition']].apply(le.fit_transform)
test[['MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition']]=test[['MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition']].apply(le.fit_transform)
train
#df = pd.get_dummies(train, columns=['MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-32-32b8ec33d7d5>:9: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.fillna(train.mean(), inplace=True)
<ipython-input-32-32b8ec33d7d5>:10: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  test.fillna(test.mean(), inplace=True)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,3,65.0,8450,1,2,3,3,0,4,...,0,3,4,4,0,2,2008,8,4,208500
1,20,3,80.0,9600,1,2,3,3,0,2,...,0,3,4,4,0,5,2007,8,4,181500
2,60,3,68.0,11250,1,2,0,3,0,4,...,0,3,4,4,0,9,2008,8,4,223500
3,70,3,60.0,9550,1,2,0,3,0,0,...,0,3,4,4,0,2,2006,8,0,140000
4,60,3,84.0,14260,1,2,0,3,0,2,...,0,3,4,4,0,12,2008,8,4,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,1,2,3,3,0,4,...,0,3,4,4,0,8,2007,8,4,175000
1456,20,3,85.0,13175,1,2,3,3,0,4,...,0,3,2,4,0,2,2010,8,4,210000
1457,70,3,66.0,9042,1,2,3,3,0,4,...,0,3,0,2,2500,5,2010,8,4,266500
1458,20,3,68.0,9717,1,2,3,3,0,4,...,0,3,4,4,0,4,2010,8,4,142125


In [33]:

# Split data into features and target
train_X = train.iloc[:, :-1]  # All columns except last one (SalePrice)
train_y = train.iloc[:, -1]  # SalePrice column

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# Train model
model = LinearRegression()
model.fit(train_X, train_y)

LinearRegression()

In [35]:
# Make predictions on test data
test_X = test
predictions = model.predict(test_X)

In [37]:
import numpy as np
# Evaluate model
rmse = np.sqrt(mean_squared_error(train_y, model.predict(train_X)))
r2 = r2_score(train_y, model.predict(train_X))

print("RMSE: ", rmse)
print("R2 score: ", r2)

RMSE:  30246.719624421326
R2 score:  0.8549397945313106
